In [1]:
import tensorflow as tf
import tensorflow.keras as keras

import pandas as pd

import numpy as np

In [2]:
hidden_dim = 2
output_dim = 2
Tomita = 3
thred = 0.05
trpath = '/home/guo/code/test/RNNRE/data/tomita/T' + str(Tomita) + '_train'
tepath_prefix = '/home/guo/code/test/RNNRE/data/tomita/T' + str(Tomita) + '_test'
testn = 4
inpalpha = {'s': 2, 'e': 3, '#': 3, '0': 0, '1': 1}
labalpha = {'0': 0, '1': 1}
ts = [32, 64, 128, 256]
RNN = keras.layers.SimpleRNN

In [3]:
train_df = pd.read_csv(trpath, dtype={1: str}, index_col=None, header=None)
train_lx = train_df.iloc[:, 0].values.tolist()
train_ly = train_df.iloc[:, 1].values.tolist()
train_x = [list(map(lambda x: inpalpha[x], i)) for i in train_lx]
train_y = [list(map(lambda x: labalpha[x], i)) for i in train_ly]
trx_np = np.array(train_x)
try_np = np.array(train_y)
trx = tf.one_hot(trx_np, 4)
trl = tf.one_hot(try_np, 2)

tex_l = list()
tel_l = list()
for i in range(testn):
    test_df = pd.read_csv(tepath_prefix + str(i+1), dtype={1: str}, index_col=None, header=None)
    test_lx = test_df.iloc[:, 0].values.tolist()
    test_ly = test_df.iloc[:, 1].values.tolist()
    test_x = [list(map(lambda x: inpalpha[x], i)) for i in test_lx]
    test_y = [list(map(lambda x: labalpha[x], i)) for i in test_ly]
    tex_np = np.array(test_x)
    tey_np = np.array(test_y)
    tex = tf.one_hot(tex_np, 4)
    tel = tf.one_hot(tey_np, 2)
    tex_l.append(tex)
    tel_l.append(tel)

In [ ]:
model = keras.Sequential([RNN(hidden_dim, input_shape=(32, 4), return_sequences=True),
                         keras.layers.Dense(output_dim),
                         keras.layers.Activation('softmax')])

model.compile(optimizer='rmsprop',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
his = model.fit(trx, trl, steps_per_epoch=200, epochs=1)
while his.history['loss'][0] > thred:
    his = model.fit(trx, trl, steps_per_epoch=200, epochs=1)


for timestep, tex, tel in zip(ts, tex_l, tel_l):
    emodel = keras.Sequential([RNN(hidden_dim, input_shape=(timestep, 4), return_sequences=True),
                             keras.layers.Dense(output_dim),
                             keras.layers.Activation('softmax')])
    emodel.set_weights(model.get_weights())
    emodel.compile(optimizer='rmsprop',
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])
    a = emodel.evaluate(tex, tel, steps=5)
    print(a)

Epoch 1/1
200/200 [==============================] - 86s 429ms/step - loss: 0.7070 - acc: 0.4795
Epoch 1/1
200/200 [==============================] - 79s 395ms/step - loss: 0.6728 - acc: 0.5680
Epoch 1/1
200/200 [==============================] - 76s 378ms/step - loss: 0.6477 - acc: 0.6092
Epoch 1/1
200/200 [==============================] - 76s 381ms/step - loss: 0.6285 - acc: 0.6627
Epoch 1/1
200/200 [==============================] - 74s 372ms/step - loss: 0.6154 - acc: 0.6740
Epoch 1/1
200/200 [==============================] - 76s 378ms/step - loss: 0.6023 - acc: 0.6774
Epoch 1/1
200/200 [==============================] - 75s 376ms/step - loss: 0.5954 - acc: 0.6940
Epoch 1/1
200/200 [==============================] - 75s 374ms/step - loss: 0.5887 - acc: 0.6969
Epoch 1/1
200/200 [==============================] - 75s 373ms/step - loss: 0.5797 - acc: 0.6797
Epoch 1/1
200/200 [==============================] - 74s 369ms/step - loss: 0.5722 - acc: 0.6723
Epoch 1/1
200/200 [===========

200/200 [==============================] - 75s 374ms/step - loss: 0.3792 - acc: 0.8405
Epoch 1/1
200/200 [==============================] - 74s 369ms/step - loss: 0.3791 - acc: 0.8405
Epoch 1/1
200/200 [==============================] - 73s 367ms/step - loss: 0.3789 - acc: 0.8410
Epoch 1/1
200/200 [==============================] - 72s 359ms/step - loss: 0.3788 - acc: 0.8415
Epoch 1/1
200/200 [==============================] - 74s 368ms/step - loss: 0.3787 - acc: 0.8415
Epoch 1/1
200/200 [==============================] - 72s 362ms/step - loss: 0.3785 - acc: 0.8415
Epoch 1/1
200/200 [==============================] - 74s 372ms/step - loss: 0.3784 - acc: 0.8415
Epoch 1/1
200/200 [==============================] - 72s 358ms/step - loss: 0.3783 - acc: 0.8415
Epoch 1/1
200/200 [==============================] - 75s 373ms/step - loss: 0.3781 - acc: 0.8414
Epoch 1/1
200/200 [==============================] - 74s 372ms/step - loss: 0.3780 - acc: 0.8414
Epoch 1/1
200/200 [=====================

200/200 [==============================] - 71s 357ms/step - loss: 0.3685 - acc: 0.8423
Epoch 1/1
200/200 [==============================] - 74s 370ms/step - loss: 0.3684 - acc: 0.8423
Epoch 1/1
200/200 [==============================] - 73s 364ms/step - loss: 0.3684 - acc: 0.8423
Epoch 1/1
200/200 [==============================] - 74s 372ms/step - loss: 0.3683 - acc: 0.8424
Epoch 1/1
200/200 [==============================] - 71s 353ms/step - loss: 0.3682 - acc: 0.8424
Epoch 1/1
200/200 [==============================] - 69s 345ms/step - loss: 0.3682 - acc: 0.8424
Epoch 1/1
200/200 [==============================] - 74s 368ms/step - loss: 0.3681 - acc: 0.8424
Epoch 1/1
200/200 [==============================] - 74s 372ms/step - loss: 0.3681 - acc: 0.8424
Epoch 1/1
200/200 [==============================] - 75s 374ms/step - loss: 0.3680 - acc: 0.8424
Epoch 1/1
200/200 [==============================] - 75s 374ms/step - loss: 0.3679 - acc: 0.8424
Epoch 1/1
200/200 [=====================

200/200 [==============================] - 67s 335ms/step - loss: 0.3652 - acc: 0.8398
Epoch 1/1
200/200 [==============================] - 67s 333ms/step - loss: 0.3652 - acc: 0.8398
Epoch 1/1
200/200 [==============================] - 63s 317ms/step - loss: 0.3651 - acc: 0.8398
Epoch 1/1
200/200 [==============================] - 62s 308ms/step - loss: 0.3651 - acc: 0.8398
Epoch 1/1
200/200 [==============================] - 65s 323ms/step - loss: 0.3651 - acc: 0.8398
Epoch 1/1
200/200 [==============================] - 63s 317ms/step - loss: 0.3650 - acc: 0.8397
Epoch 1/1
200/200 [==============================] - 62s 308ms/step - loss: 0.3650 - acc: 0.8398
Epoch 1/1
200/200 [==============================] - 62s 308ms/step - loss: 0.3650 - acc: 0.8398
Epoch 1/1
200/200 [==============================] - 65s 323ms/step - loss: 0.3650 - acc: 0.8398
Epoch 1/1
200/200 [==============================] - 66s 330ms/step - loss: 0.3649 - acc: 0.8398
Epoch 1/1
200/200 [=====================

200/200 [==============================] - 65s 323ms/step - loss: 0.3612 - acc: 0.8424
Epoch 1/1
200/200 [==============================] - 69s 347ms/step - loss: 0.3611 - acc: 0.8426
Epoch 1/1
200/200 [==============================] - 61s 307ms/step - loss: 0.3610 - acc: 0.8427
Epoch 1/1
200/200 [==============================] - 63s 316ms/step - loss: 0.3610 - acc: 0.8428
Epoch 1/1
200/200 [==============================] - 64s 320ms/step - loss: 0.3609 - acc: 0.8430
Epoch 1/1
200/200 [==============================] - 65s 324ms/step - loss: 0.3608 - acc: 0.8431
Epoch 1/1
200/200 [==============================] - 63s 313ms/step - loss: 0.3607 - acc: 0.8431
Epoch 1/1
200/200 [==============================] - 62s 311ms/step - loss: 0.3606 - acc: 0.8431
Epoch 1/1
200/200 [==============================] - 63s 317ms/step - loss: 0.3605 - acc: 0.8432
Epoch 1/1
200/200 [==============================] - 63s 315ms/step - loss: 0.3604 - acc: 0.8433
Epoch 1/1
200/200 [=====================

200/200 [==============================] - 71s 357ms/step - loss: 0.3497 - acc: 0.8527
Epoch 1/1
200/200 [==============================] - 61s 304ms/step - loss: 0.3496 - acc: 0.8528
Epoch 1/1
200/200 [==============================] - 71s 354ms/step - loss: 0.3494 - acc: 0.8529
Epoch 1/1
200/200 [==============================] - 71s 354ms/step - loss: 0.3492 - acc: 0.8531
Epoch 1/1
200/200 [==============================] - 71s 356ms/step - loss: 0.3490 - acc: 0.8532
Epoch 1/1
200/200 [==============================] - 72s 358ms/step - loss: 0.3488 - acc: 0.8533
Epoch 1/1
200/200 [==============================] - 73s 363ms/step - loss: 0.3487 - acc: 0.8534
Epoch 1/1
200/200 [==============================] - 70s 349ms/step - loss: 0.3485 - acc: 0.8535
Epoch 1/1
200/200 [==============================] - 65s 324ms/step - loss: 0.3483 - acc: 0.8536
Epoch 1/1
200/200 [==============================] - 67s 333ms/step - loss: 0.3481 - acc: 0.8536
Epoch 1/1
200/200 [=====================

200/200 [==============================] - 73s 364ms/step - loss: 0.3367 - acc: 0.8610
Epoch 1/1
200/200 [==============================] - 66s 330ms/step - loss: 0.3364 - acc: 0.8611
Epoch 1/1
200/200 [==============================] - 65s 323ms/step - loss: 0.3361 - acc: 0.8612
Epoch 1/1
200/200 [==============================] - 63s 316ms/step - loss: 0.3357 - acc: 0.8614
Epoch 1/1
200/200 [==============================] - 65s 323ms/step - loss: 0.3354 - acc: 0.8634
Epoch 1/1
200/200 [==============================] - 64s 318ms/step - loss: 0.3350 - acc: 0.8636
Epoch 1/1
200/200 [==============================] - 61s 306ms/step - loss: 0.3347 - acc: 0.8637
Epoch 1/1
200/200 [==============================] - 63s 314ms/step - loss: 0.3343 - acc: 0.8640
Epoch 1/1
200/200 [==============================] - 64s 321ms/step - loss: 0.3339 - acc: 0.8642
Epoch 1/1
200/200 [==============================] - 63s 313ms/step - loss: 0.3335 - acc: 0.8644
Epoch 1/1
200/200 [=====================